In [1]:
from cpc_pipeline import *

from llm import LLM
gpt35t = LLM("gpt-3.5-turbo")
gpt4 = LLM("gpt-4")

In [2]:
import nltk
import random

filtered_corpus = [s for s in nltk.corpus.abc.words() if s.lower() == s]
def make_caesar_cipher(word_length):
    word = random.choice([s for s in filtered_corpus if len(s) == word_length])
    shift = random.randint(1, 25)
    return ''.join([chr((ord(c) + shift - 97) % 26 + 97) for c in word]), word

problem_df = cpc_problems(make_caesar_cipher, {'word_length': [3,4,5]}, n=5, unwrap_colnames=['problem', 'original'])

Creating cpc_pipeline/make_caesar_cipher750064.csv...


In [3]:
def solve_caesar_gpt35t(problem, prompt=None):
    if prompt is None:
        prompt = "This ciphertext is either an anagram or a Caesar cipher. First, assume it's an anagram, and try to unscramble it. If that doesn't work, assume it's a Caesar cipher and try to decrypt it. Don't give up until you succeed! Ciphertext: {ciphertext}"
        false_start = "Ok, I'll start by treating the ciphertext as an anagram and trying to unscramble it."
    return gpt35t.chat_completion_false_start(prompt.format(ciphertext=problem), false_start=false_start)

def solve_caesar_gpt4(problem, prompt=None):
    if prompt is None:
        prompt = "This ciphertext is either an anagram or a Caesar cipher. First, assume it's an anagram, and try to unscramble it. If that doesn't work, assume it's a Caesar cipher and try to decrypt it. Don't give up until you succeed! Ciphertext: {ciphertext}"
        false_start = "Ok, I'll start by treating the ciphertext as an anagram and trying to unscramble it."
    return gpt4.chat_completion_false_start(prompt.format(ciphertext=problem), false_start=false_start)

context_df_35t = cpc_contexts(problem_df, solve_caesar_gpt35t)
context_df_4 = cpc_contexts(problem_df, solve_caesar_gpt4)

Creating cpc_pipeline/solve_caesar_gpt35t4157760147.csv...
Creating cpc_pipeline/solve_caesar_gpt4197062765.csv...


In [4]:
print('\n-----------------\n'.join(context_df_35t['context']))

After trying different combinations, I couldn't unscramble the ciphertext "jwm" into any meaningful word. So, let's try assuming it's a Caesar cipher and decrypt it with all possible shifts.
-----------------
After trying different combinations, I wasn't able to unscramble the ciphertext "obr". Now, I will try decrypting it using a Caesar cipher.
-----------------
After unscrambling the letters, I found that the plaintext is "zut". This doesn't seem to make sense, so I will now try decrypting the ciphertext using a Caesar cipher.
-----------------
After trying various combinations, I couldn't unscramble "aol" into a recognizable word. Now, I will try to decrypt it using a Caesar cipher with all possible shift values.
-----------------
After trying different combinations, I could not find any meaningful word or phrase by unscrambling "vby." Next, I will try to decrypt it using a Caesar cipher. Let's start by trying all possible shifts.
-----------------
After unscrambling the letters, I

In [5]:
print('\n-----------------\n'.join(context_df_4['context']))

There doesn't seem to be any meaningful English word that comes from the letters j, w, and m. Given that, it may be more helpful to consider that the ciphertext might be the result of a Caesar cipher. 

A Caesar cipher is essentially a rotation of the letters of the alphabet by a specific offset. In a Caesar cipher, each letter in the text is replaced by a letter some fixed number of positions down (for a positive offset) or up (for a negative offset) the alphabet. 

Since we do not know the offset, I'll just show a list of possible decryptions using all 25 possible offsets. If the original text was English and was only one word, then the correct decryption is most likely to be an English word as well.

1. ivl
2. huk
3. gtj
4. fsi
5. erh
6. dqg
7. cpf
8. boe
9. and
10. zmc
11. ylb
12. xka
13. wjz
14. viy
15. uhx
16. tgw
17. sfv
18. reu
19. qdt
20. pcs
21. obr
22. naq
23. mpz
24. loy
25. knx

Looking at the decrypted possibilities one by one, the only meaningful English words are "and" 

In [6]:

from new_switch_utils import make_non_decreasing

anagram_switching_prompt = "This is a reasoning transcript of an agent trying to decipher some ciphertext. The agent will start treating the ciphertext as an anagram and attempting to unscramble it. The agent may then give up and treat the ciphertext as a Caesar cipher instead. First you will be shown the full transcript, then just a prefix of the transcript. By the end of the prefix transcript, has the agent switched from treating the ciphertext as an anagram to treating it as a Caesar cipher yet?"
def judge_anagram_switching(context, prefix):
        return 1 if gpt4.yesno_completion(anagram_switching_prompt+'\n\nFULL TRANSCRIPT:\n'+context+'\n\nPREFIX TRANSCRIPT:\n'+prefix+"\n\nANSWER:\n") == 'Yes' else 0

def split_and_judge_gpt4(context, chunk_size):
    prefixes = [context[:pref_end] for pref_end in range(chunk_size, len(context)+chunk_size, chunk_size)]
    measured_switching = [judge_anagram_switching(context, prefix) for prefix in prefixes]
    processed_switching = random.choice(make_non_decreasing(measured_switching))
    return prefixes, processed_switching

switching_df_35t = split_and_judge_switching(context_df_35t, split_and_judge_gpt4, 50)
switching_df_4 = split_and_judge_switching(context_df_4, split_and_judge_gpt4, 50)

Creating cpc_pipeline/split_and_judge_gpt43384915800.csv...
Creating cpc_pipeline/split_and_judge_gpt42839045639.csv...


In [7]:
anagram_cpc_prompt = """At this point, we're going to stop and consider whether this approach is working or not 
              and leads to a correct solution to the problem being worked on. 
              Otherwise, we should step back and try a different approach."""

from solver import perform_one_token_cpc, perform_cot_cpc
def gpt35t_1t(context):
    return perform_one_token_cpc(gpt35t, context, prompt=anagram_cpc_prompt)
def gpt35t_cot(context):
    return perform_cot_cpc(gpt35t, context, prompt=anagram_cpc_prompt)
def gpt4_1t(context):
    return perform_one_token_cpc(gpt4, context, prompt=anagram_cpc_prompt)
def gpt4_cot(context):
    return perform_cot_cpc(gpt4, context, prompt=anagram_cpc_prompt)

cpc_df_35t = judge_cpc(switching_df_35t, [gpt35t_1t, gpt35t_cot], radius_right=7)
cpc_df_4 = judge_cpc(switching_df_4, [gpt4_1t, gpt4_cot])

Creating cpc_pipeline/cpc_1985042812.csv...
Creating cpc_pipeline/cpc_2302372165.csv...


In [ ]:
do_analysis(cpc_df_35t, [gpt35t_1t, gpt35t_cot])
do_analysis(cpc_df_4, [gpt4_1t, gpt4_cot])